<a href="https://colab.research.google.com/github/k4zuy/Rep-Baal-Progress/blob/main/baal_testground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install baal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Imports

In [ ]:
import argparse
import random
import csv
from copy import deepcopy
from time import time
import datetime
from dataclasses import dataclass
import numpy as np
import PIL

import torch
import torch.backends
from torch import optim
from torch.hub import load_state_dict_from_url
from torch.nn import CrossEntropyLoss
from torchvision import datasets
from torchvision.models import vgg16
from torchvision.transforms import transforms
from tqdm import tqdm
from torch import nn
import torch.nn.functional as F

from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.active_loop import ActiveLearningLoop
from baal.bayesian.dropout import patch_module
from baal import ModelWrapper
from baal.utils.metrics import Accuracy
from baal.active.heuristics import BALD
from baal.active.dataset import ActiveLearningDataset

import aug_lib

from ExtendedActiveLearningDataset import ExtendedActiveLearningDataset

In [ ]:
@dataclass
class ExperimentConfig:
    epoch: int = 2
    batch_size: int = 32
    initial_pool: int = 1000
    query_size: int = 100
    lr: float = 0.001
    heuristic: str = 'bald'
    iterations: int = 40
    training_duration: int = 10

In [ ]:
def get_datasets(initial_pool, n_augmentations):
    transform = transforms.Compose(
        [
            # transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(3 * [0.5], 3 * [0.5]),
        ]
    )
    aug_transform = transforms.Compose(
        [
            # transforms.Resize((224, 224)),
            aug_lib.TrivialAugment(),
            transforms.ToTensor(),
            transforms.Normalize(3 * [0.5], 3 * [0.5]),
        ]
    )
    test_transform = transforms.Compose(
        [
            # transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(3 * [0.5], 3 * [0.5]),
        ]
    )
    # Note: We use the test set here as an example. You should make your own validation set.
    train_ds = datasets.CIFAR10(
        ".", train=True, transform=transform, target_transform=None, download=True
    )

    aug_train_ds = datasets.CIFAR10(
        ".", train=True, transform=aug_transform, target_transform=None, download=True
    )
    test_set = datasets.CIFAR10(
        ".", train=False, transform=test_transform, target_transform=None, download=True
    )
    eald_set = ExtendedActiveLearningDataset(train_ds)

    # active_set = ActiveLearningDataset(
    #    train_ds, pool_specifics={"transform": test_transform}
    # )
    eald_set.augment_n_times(n_augmentations, augmented_dataset=aug_train_ds)
    # We start labeling randomly.
    eald_set.label_randomly(initial_pool)
    return eald_set, test_set

In [ ]:
hyperparams = ExperimentConfig()
use_cuda = torch.cuda.is_available()
torch.backends.cudnn.benchmark = True
random.seed(1337)
torch.manual_seed(1337)
if not use_cuda:
    print("warning, the experiments would take ages to run on cpu")

# Get datasets
active_set, test_set = get_datasets(hyperparams.initial_pool,2)

# Get our model.
heuristic = get_heuristic(hyperparams.heuristic)
criterion = CrossEntropyLoss()
model = vgg16(num_classes=10)

# change dropout layer to MCDropout
model = patch_module(model)

if use_cuda:
    model.cuda()
optimizer = optim.SGD(model.parameters(), lr=hyperparams.lr, momentum=0.9)

# Wraps the model into a usable API.
model = ModelWrapper(model, criterion)

# for ActiveLearningLoop we use a smaller batchsize
# since we will stack predictions to perform MCDropout.
active_loop = ActiveLearningLoop(active_set,
                                 model.predict_on_dataset,
                                 heuristic,
                                 hyperparams.query_size,
                                 batch_size=1,
                                 iterations=hyperparams.iterations,
                                 use_cuda=use_cuda,
                                 verbose=False)

# We will reset the weights at each active learning step so we make a copy.
init_weights = deepcopy(model.state_dict())

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
orig len50000
augmented n times0


In [ ]:
active_set.n_labelled

3000

In [ ]:
active_set.n_unlabelled

147000

In [ ]:
pool = active_set.pool
indices = np.arange(len(pool))
indices

array([     0,      1,      2, ..., 146997, 146998, 146999])

In [ ]:
probs = model.predict_on_dataset(pool,batch_size=10,iterations=20,use_cuda=use_cuda)

[79-MainThread   ] [baal.modelwrapper:predict_on_dataset_generator:263] 2022-11-23T17:29:56.501458Z [info     ] Start Predict                  dataset=147000
100%|██████████| 14700/14700 [10:13<00:00, 23.96it/s]


In [ ]:
to_label, uncertainty = heuristic.get_ranks(probs) # to_label -> 
to_label = indices[np.array(to_label)]

In [ ]:
uncertainty

array([7.1525574e-05, 1.0538101e-04, 1.2636185e-04, ..., 1.6140938e-04,
       1.6164780e-04, 7.8678131e-05], dtype=float32)

In [ ]:
# größte uncertainty 
# (ix_global_img sowieso in to_label[0] aber for-Schleife für besseres Verständnis)
# also to_label[0] == uncertainty[ix_global_img]
u = 0
ix_pool_img = 0
for i in range(len(uncertainty)):
  if(uncertainty[i]>u): 
    u = uncertainty[i]
    ix_pool_img = i

print("größte uncertainty: " + str(u))
print("dazugehöriger Index: " + str(ix_pool_img))

größte uncertainty: 0.0004014969
dazugehöriger Index: 104559


In [ ]:
to_label

array([104559, 131009,  78579, ...,  88029,  65177,  90586])

In [ ]:
# größter index
l = 0
for i in range(len(to_label)):
  if(to_label[i]>l): l = to_label[i]

l

146999

In [ ]:
oracle_set_one = active_set._pool_to_oracle_index([104559])
print(oracle_set_one)
oracle_set_two = active_set._pool_to_oracle_index(to_label)
print(oracle_set_two[0])
#ix_global_img = oracle_set[0]
#ix_global_img


[106697]
106697


In [ ]:
oracle_set = active_set._pool_to_oracle_index(pool)
ix_global_img = oracle_set[ix_pool_img]

In [ ]:
pool_img = active_set.pool[ix_pool_img]
global_img = active_set._dataset[ix_global_img]
#global_img = active_set._dataset[ix_global_img]

In [ ]:
print(pool_img)
print(global_img)

(tensor([[[ 0.4039,  0.2784,  0.2627,  ...,  0.5529,  0.4824,  0.3725],
         [ 0.4745,  0.4588,  0.4039,  ...,  0.8902,  0.9137,  0.8824],
         [ 0.4824,  0.5529,  0.3961,  ...,  1.0000,  1.0000,  1.0000],
         ...,
         [-0.6314, -0.4510, -0.3490,  ..., -1.0000, -0.8980, -0.7333],
         [-0.5922, -0.4667, -0.2078,  ..., -1.0000, -1.0000, -0.8745],
         [-0.5686, -0.3961,  0.2392,  ..., -1.0000, -1.0000, -1.0000]],

        [[ 0.4353,  0.2863,  0.2392,  ...,  0.5922,  0.5216,  0.4196],
         [ 0.5059,  0.4588,  0.3804,  ...,  0.9294,  0.9373,  0.9059],
         [ 0.5059,  0.5529,  0.3569,  ...,  1.0000,  1.0000,  1.0000],
         ...,
         [-0.4510, -0.3020, -0.2235,  ..., -0.9529, -0.8118, -0.6549],
         [-0.4353, -0.3176, -0.0980,  ..., -1.0000, -0.9294, -0.7882],
         [-0.3961, -0.2471,  0.3490,  ..., -1.0000, -1.0000, -0.9451]],

        [[ 0.5059,  0.3333,  0.2863,  ...,  0.7098,  0.6157,  0.4824],
         [ 0.5922,  0.5294,  0.4353,  ...,  

Versuche herauszufinden was folgende Zeile genau macht und wieso aber bisher erfolglos

In [ ]:
to_label_two = indices[np.array(to_label)]
to_label_two

array([104559, 131009,  78579, ...,  88029,  65177,  90586])

In [ ]:
print(type(to_label))
print(type(to_label_two))
print(len(to_label))
print(len(to_label_two))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
147000
147000


In [ ]:
indices[np.array(to_label)]

array([104559, 131009,  78579, ...,  88029,  65177,  90586])

In [ ]:
t = np.array([3,7,1,4])
t

array([3, 7, 1, 4])